In [ ]:
! pip install --upgrade google-cloud-bigquery-storage

In [1]:
import google, numpy as np, pandas as pd, geopandas as gpd
from google.cloud import aiplatform, bigquery
from google.cloud.bigquery_storage import BigQueryReadClient, types
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials = cred, project = proj)

In [2]:
yr = 2017
state_abbr = 'RI'
proj_id = 'cmat-315920'

In [3]:
query_str = f"""
select
    state_fips_code as fips
    , state_postal_abbreviation as abbr
    , state_name
from
    bigquery-public-data.census_utility.fips_codes_states
"""
states = bqclient.query(query_str).result().to_dataframe()
state = states[states['abbr']==state_abbr].iloc[0]
state

fips                    44
abbr                    RI
state_name    Rhode Island
Name: 39, dtype: object

In [10]:
# block equivalency (matches block to US congressional district)
def congress_to_yr(congress):
    return 1786 + 2 * congress

def yr_to_congress(yr):
    return int((yr-1786)/2)

def geo_id_decompose(geo_id):
    #https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
    state

def get_block_equivalency(yr):
    congress = yr_to_congress(yr)
    query_str = f"""
        select
            *
        from (
            select
                substring(geo_id, 0, 2) as state_fips
                , substring(geo_id, 3, 3) as county_fips
                , substring(geo_id, 5, 6) as tract_ce
                , substring(geo_id, 11, 1) as blockgroup_ce
                , cd
                , rank() over (partition by geo_id order by n desc) as r
            from (
                select
                    left(BLOCKID, 12) as geo_id
                    , CD{congress} as cd    
                    , count(*) as n
                from 
                    {proj_id}.Block_Equivalency_Files.{congress}th_BEF
                group by
                    1, 2
                ) as A
            ) as B
        where
            r = 1
        """
    blocks = bqclient.query(query_str).result().to_dataframe()
    return blocks
cd = get_block_equivalency(yr)
cd.head(2)

,state_fips,county_fips,tract_ce,blockgroup_ce,cd,r
0,01,003,301160,2,01,1
1,01,007,701000,3,06,1


In [5]:
# geo
# input is WKT in NAD83 - https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tgrshp2020/TGRSHP2020_TechDoc_Ch3.pdf
# use ESRI:102003 for area calculations # https://epsg.io/102003
# use ESRI:102005 for length calculations # https://epsg.io/102005

def get_bg_geo(fips):
    query_str = f"""
    select
        --geo_id
        state_fips_code as state_fips
        , county_fips_code as county_fips
        , tract_ce
        , blockgroup_ce
        --, lsad_name
        --, mtfcc_feature_class_code.
        --, functional_status
        --, area_land_meters
        --, area_water_meters
        --, internal_point_lat as lat
        --, internal_point_lon aas loni
        --, internal_point_geom
        , blockgroup_geom as geometry
    from
        bigquery-public-data.geo_census_blockgroups.blockgroups_{fips}
    """
    df = bqclient.query(query_str).result().to_dataframe()
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    return gpd.GeoDataFrame(df, geometry='geometry', crs='NAD83')


# def compute_area(geo, col=None):
#     if col:
#         geo = geo.
#     return geo.to_crs('ESRI:102003').area   / (1000**2)

def compute_perim(geo):
    return geo.to_crs('ESRI:102005').length / 1000

def compute_area_union(geo):
    geo.unary_union.to_crs('ESRI:102003').area   / (1000**2)


geo = get_bg_geo(state['fips'])
geo.head(2)

,geo_id,state_fips,county_fips,tract_ce,blockgroup_ce,geometry
0,440010303001,44,001,030300,1,"POLYGON ((-71.30076 41.74358, -71.30087 41.743..."
1,440010302001,44,001,030200,1,"POLYGON ((-71.33266 41.76572, -71.33235 41.765..."


In [11]:
# acs

def get_acs(yr):
    query_str = f"""
    select
        substring(geo_id, 0, 2) as state_fips
        , substring(geo_id, 3, 3) as county_fips
        , substring(geo_id, 5, 6) as tract_ce
        , substring(geo_id, 11, 1) as blockgroup_ce
        , total_pop as pop
    from
        bigquery-public-data.census_bureau_acs.blockgroup_{yr}_5yr
    """
    df = bqclient.query(query_str).result().to_dataframe()
    return df
acs = get_acs(yr)
acs.head(2)

,state_fips,county_fips,tract_ce,blockgroup_ce,pop
0,39,165,503160,0,500.0
1,06,115,504090,2,50.0


In [7]:
# centroids

def get_centroids(fips):
    query_str = f"""
    select
        STATEFP as state_fips
        , COUNTYFP AS county_fips
        , TRACTCE as tract_ce
        , BLKGRPCE as blockgroup_ce
        , POPULATION as pop
        , LATITUDE as lat
        , LONGITUDE as lon
    from
        {proj_id}.BLOCK_CENTROIDS.block_centroid_{fips}
    """
    df = bqclient.query(query_str).result().to_dataframe()
#     df.columns = ['state_fips', 'countyfips', 'tract_ce', 'blockkgroup_ce', 'pop',  	latitude 	longitude
    
#     df.columns = [x.lower() for x in df.columns]
    
    return df
centroids = get_centroids(state['fips'])
display(centroids.head(2))

,state_fips,county_fips,tract_ce,blockgroup_ce,pop,lat,lon
0,44,7,12801,3,2138,42.004282,-71.571653
1,44,7,12801,3,2138,42.004282,-71.571653


In [14]:
for df in [cd, geo, acs, centroids]:
    display(df.head(2))
    display(df.shape)


,state_fips,county_fips,tract_ce,blockgroup_ce,cd,r
0,01,003,301160,2,01,1
1,01,007,701000,3,06,1


(220486, 6)

,geo_id,state_fips,county_fips,tract_ce,blockgroup_ce,geometry
0,440010303001,44,001,030300,1,"POLYGON ((-71.30076 41.74358, -71.30087 41.743..."
1,440010302001,44,001,030200,1,"POLYGON ((-71.33266 41.76572, -71.33235 41.765..."


(815, 6)

,state_fips,county_fips,tract_ce,blockgroup_ce,pop
0,39,165,503160,0,500.0
1,06,115,504090,2,50.0


(220333, 5)

,state_fips,county_fips,tract_ce,blockgroup_ce,pop,lat,lon
0,44,7,12801,3,2138,42.004282,-71.571653
1,44,7,12801,3,2138,42.004282,-71.571653


(1630, 7)

In [ ]:
acs.head(2)

In [ ]:
geo.head(2)

In [ ]:
geo.to_crs('ESRI:102005').length / 1000

In [ ]:
geo.u

In [ ]:
import geopandas as gpd
gdf = get_bg_shapes('44')
# gdf.plot()
A = gdf.to_crs('ESRI:102003').area / 1000
A.sum()

In [ ]:
import plotly.express as px

fig = px.choropleth(gdf, geojson=gdf.geometry,
                    color="area_land_meters",
                    locations=gdf.index,
#                     featureidkey="properties.district",
#                     projection="mercator"
                   )

fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
gdf.columns